### 資料轉換

- positive skew 正偏態或右偏態通常會作log轉換消除偏態(Gaussian-like)  
- negative skew 負偏態或左偏態通常會作exp指數轉換消除偏態(Gaussian-like)  
- Min-max scaling 使資料所有features都linearly scale到[-1,1]或[0,1]
- Standardize  使資料所有features都scale到~N(0,1)  
- Robust scaling 因為有outlier使linearly scale到[-1,1]或[0,1]時導致資料偏移, 這時會用四分位數中第一分位數跟第三分位數距離來除避免outlier值太大  
- Thresholding 通常需要專家領域的人來說明  

- 如果沒有所有features都scale到[-1,1]或[0,1],KNN計算距離會受特定維度影響  
  For KNN, scaling prevents the distance scores being dominated by a few features
- 如果沒有所有features都scale到[-1,1]或[0,1], 作Gradient Decent往極值移動時會受特定維度影響  
  For linear regression, logistic regression, SVM, scaling helps the optimization  
- For decision trees and random forest, scaling or not is not important

### Imbalanced classification 
- Under-sample the majority class  
- Over-sample the minority class  

### Selecting Hyper-parameters  
將資料切成training data跟testing data  
將training data切成k fold個validation data  
假設k為5, 則拿4份當作training data, 1份當作validation data  
拿training data先用不同的Hyper-parameters參數$\lambda$訓練, 再用validation data檢驗  
取不同份validation data 重複k次計算validation data檢驗的誤差平均  
就可以得出最合適的Hyper-parameters參數$\lambda$  
再用這組Hyper-parameters參數$\lambda$重新訓練training data加上validation data的資料  
得到的模型來預測testing data  

**選擇要比較的$\lambda$的方法** 
- grid search  
- random search(效果較佳)  
- Bayesian optimization  
 - Pick strategy: tradeoff between  
   1. Exploration  
   Hyper-parameters for which the outcome is most uncertain  
   2. Exploitation  
   Hyper-parameters which are expected to have a good outcome  
   
 
   

### Multi-class classification  
- one-vs.-rest  
Target Label = {Red,Green,Blue}  
$f_1$: Red or "Not Red"  
$f_2$: Green or "Not Green"  
$f_3$: Blue or "Not Blue"  
$\hat y=arg~max_{k{\in\lbrace1,2,3\rbrace}}f_k(x_i)$
- one-vs.-one  
假設有k類, 訓練C(k,2)個分類器  
預測時同時將資料餵給C(k,2)個分類器,大多數說該類別+1即預測該類別    

### Bias vs Variance  
- Bias通常來自對model的估計偏差, 比如指數型估計成線性型, model太小無法涵蓋真實問題  
- Variance通常來自訓練資料集的不均勻, 相對母體來說偏向某一部分, training data太小無法涵蓋整個母體  


### Model complexity vs Error  
- 針對high bias, low varance我們會先固定訓練資料集且增加model複雜度, 這時prediction error會下降  
  當bias變小(model複雜度足夠), varance變大, 這時prediction error會不降反升  
- high varance我們會嘗試增加data 

### Data Size vs Model complexity  
1. training data分得好嗎? 分得不好增加model複雜度, 分得好跳到2  
2. training data分好後, 看testing data分得好嗎? 分得不好增加training data size再回到1 